In [1]:
import numpy as np
import pandas as pd
import collections
import gym
import matplotlib.pyplot as plt
from lib import data, environ
from typing import List, Optional, Tuple, Any
from tensorforce import Agent, Environment
from tensorforce.core.networks import AutoNetwork
from tensorforce.execution import Runner
from tensorforce.core.layers import Dense, Gru
%load_ext blackcellmagic

In [16]:
# prices = data.load_relative("data/BTC_history_n.csv")
prices = data.load_by_date("data/BTC_history_n.csv", date_range=('2018-01-01', '2019-12-31'))

In [17]:
env = environ.StocksEnv(
    prices={"btc": prices}, state_1d=False, volumes=True, commission=0.0, bars_count=10, reward_on_close=False, reset_on_close=True,
)
obs = env.reset()
obs, reward, done, info = env.step(0)
print(f"Reward: {reward}")
print(f"Done: {done}")
print(f"Info: {info}")
print(f'Number of trading days in data: {env._prices.get("btc").open.shape[0]}')

Reward: 0.0
Done: False
Info: {'instrument': 'btc', 'offset': 92}
Number of trading days in data: 730


In [18]:
environment = Environment.create(environment=env, max_episode_timesteps=500)
print(environment.actions())
print(environment.states())

{'type': 'int', 'shape': (), 'num_values': 3}
{'type': 'float', 'shape': (42,), 'min_value': None, 'max_value': None}


In [19]:
# Instantiate a Tensorforce agent
if False:
    agent_ = Agent.create(
        agent="dueling_dqn",
        max_episode_timesteps=1000,
        environment=environment,
        memory=100000,
        # update=dict(unit="timesteps", batch_size=32),
        batch_size=32,
        # optimizer=dict(type="adam", learning_rate=3e-4),
        # policy=dict(network="auto"),
        # objective="policy_gradient",
        start_updating=1e4,
        # network=dict(network=[Gru(size=5, horizon=5, name='GRU_1'), Dense(size=3, name='Dense_1')]),
        # network=dict(network=[dict(type='dense', size=32), dict(type='dense', size=3)]),
        network='auto',
        # reward_estimation=dict(horizon=4, discount=0.99),
        discount=0.99,
        target_sync_frequency=1e3,
        config=dict(name="agent_007"),
        summarizer=dict(
            directory="runs/summaries",
            # list of labels, or 'all'
            summaries=["entropy", "kl-divergence", "loss", "reward", "update-norm"],
        ),
    )

In [27]:
# Instantiate a Tensorforce agent
agent = Agent.create(
    agent="dueling_dqn",
    environment=environment,
    # optimizer=dict(optimizer="adam", learning_rate=1e-3),
    # update=dict(unit="timesteps", batch_size=32),
    batch_size=32,
    # objective="policy_gradient",
    # reward_estimation=dict(horizon=5),
    max_episode_timesteps=1000,
    memory=10000,
    config=dict(name="agent_001", device='cpu'),
    summarizer=dict(
            directory="runs/summaries",
            # list of labels, or 'all'
            # summaries=["entropy", "kl-divergence", "loss", "reward", "update-norm"],
            summaries='all',
        ),
)

In [28]:
runner = Runner(
    agent=agent,
    environment=environment,
)

In [29]:
runner.run(num_episodes=500)
runner.close()

Episodes:   0%|          | 0/500 [00:00, return=0.00, ts/ep=0, sec/ep=0.00, ms/ts=0.0, agent=0.0%]

KeyboardInterrupt: 